In [2]:
import pandas as pd
import jupyter_black
import requests
from datetime import date

In [3]:
# Headers for MLB API
headers = {
    'sec-ch-ua-platform': '"macOS"',
    'Referer': 'https://www.mlb.com/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
}

# Fetch standings data
url = 'https://bdfed.stitch.mlbinfra.com/bdfed/transform-mlb-standings?&splitPcts=false&numberPcts=false&standingsView=division&sortTemplate=3&season=2025&leagueIds=103&&leagueIds=104&standingsTypes=regularSeason&contextTeamId=&teamId=&date=2025-06-27&hydrateAlias=noSchedule&favoriteTeams=119&sortDivisions=201,202,200,204,205,203&sortLeagues=103,104,115,114&sortSports=1'
response = requests.get(url, headers=headers)
json_data = response.json()

# Combine all teamRecords across divisions
team_records = []
for record in json_data['records']:
    team_records.extend(record['teamRecords'])

# Flatten all keys (including nested team/team.division/etc.)
df = pd.json_normalize(team_records, sep='_')

row = df.query('abbreviation == "LAD"').iloc[0]

# Parse variables
games_played = row['wins'] + row['losses']
division_place = int(row['divisionRank'])

# Helper for ordinal suffix
def ordinal(n):
    return f"{n}{'tsnrhtdd'[(n//10%10!=1)*(n%10<4)*n%10::4]}"

win_pct = float(row['pct']) * 100
last_10_wins = int(row['record_lastTen'].split('-')[0])
xwins = int(row['record_xWinLossSeason'].split('-')[0])

# Assemble
summary = (
    f"LOS ANGELES ({date.today():%B %d, %Y}) — After {games_played} games this season, "
    f"the Dodgers are in {ordinal(division_place)} place in the National League West division. "
    f"The team has compiled a {row['wins']}-{row['losses']} record, winning {win_pct:.0f}% of its games so far. "
    f"They've won {last_10_wins} of the last 10 and are on pace to win at least {xwins} games in the regular season."
)

print(summary)

LOS ANGELES (June 30, 2025) — After 83 games this season, the Dodgers are in 1st place in the National League West division. The team has compiled a 52-31 record, winning 63% of its games so far. They've won 8 of the last 10 and are on pace to win at least 97 games in the regular season.


[{'date': '2025-06-29',
  'totalItems': 1,
  'totalEvents': 0,
  'totalGames': 1,
  'totalGamesInProgress': 0,
  'games': [{'gamePk': 777304,
    'gameGuid': 'f4c768dd-2e2a-4bfa-ba98-ff1f498d6d4b',
    'link': '/api/v1.1/game/777304/feed/live',
    'gameType': 'R',
    'season': '2025',
    'gameDate': '2025-06-29T11:10:00-07:00',
    'officialDate': '2025-06-29',
    'status': {'abstractGameState': 'Final',
     'codedGameState': 'F',
     'detailedState': 'Final',
     'statusCode': 'F',
     'startTimeTBD': False,
     'abstractGameCode': 'F'},
    'teams': {'away': {'leagueRecord': {'wins': 53,
       'losses': 32,
       'pct': '.624'},
      'score': 5,
      'team': {'springLeague': {'id': 114,
        'name': 'Cactus League',
        'link': '/api/v1/league/114',
        'abbreviation': 'CL'},
       'allStarStatus': 'N',
       'id': 119,
       'name': 'Los Angeles Dodgers',
       'link': '/api/v1/teams/119',
       'season': 2025,
       'venue': {'id': 22,
        'name': 